<a href="https://colab.research.google.com/github/jfishe27/github-slideshow/blob/master/Intuitive_Statistics_I_Hypothesis_Tests_and_Simulation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Intuitive Statistics with SciPy


We will learn to perform statistical analysis through computer simulation. No prior statistical background is assumed.


### Contents

* Introduction to hypothesis tests
* Monte-Carlo tests
* [Permutation tests](https://colab.research.google.com/drive/1wdrTtW8AxPcwNnCjyEteGhJxRx_szbnJ)
* [The Bootstrap](https://colab.research.google.com/drive/1woji3v387VupHvnbNMgJaqlu_EChzseX)



## Hypothesis Tests

Let's try to build up the idea of a hypothesis test from first principles.


### The toss of a coin

* We often say the outcome of a coin toss, heads or tails, is random.
* What is randomness?
* Given the available information, there are no patterns that would let us predict the outcome.

### Basic Notions

Imagine we toss a coin repeatedly.

* We expect a fair coin to come up heads as often as it comes up tails and say the **Probability** of heads is $p = 0.5$
* For a weighted coin, if the probability of heads is $p$ then as the number of tosses grows, we expect a proportion $p$ of the outcomes to be heads.

This is [Frequentist Probability](https://en.wikipedia.org/wiki/Frequentist_probability).

An event's probability is the limit of its frequency as the number of trials goes to infinity.

* **Independence**: The outcome of one toss doesn't depend on the outcome of any prior tosses.

## Can you generate random numbers?

Using only your mind, try to simulate 50 independent tosses of a fair coin. Fill in the empty string below, enter "1" for heads and "0" for tails. There are 50 x's in the comment to help you get the length right.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats

#         xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx
tosses = "10100110101011110101010111111010011010101001101001"

# Validate coin tosses and covert to array
assert len(tosses) == 50 and set(tosses) <= {"0", "1"}
tosses = np.array(list(tosses), dtype=int)
tosses

## Can computers generate random numbers?

*Anyone who considers arithmetical methods of producing random digits is, of course, in a state of sin.*

John Von Neumann

* A curious fact in math:
  * some deterministic arithmetical processes can produce numbers which appear "random"
* but if we know the algorithm we can predict the next number.
* For many applications this is sufficient.

### Example

[Linear Congruential Generator](https://en.wikipedia.org/wiki/Linear_congruential_generator)

Values are generated with a simple first order recurrence

$$T_n = (aT_{n-1} + c)\: \% \: \mathrm{modulus}$$

for integer constants $a$, $c$, and $\mathrm{modulus}$.

The initial value $T_0$ is called the $\mathrm{seed}$.

NumPy uses a related algorithm called the [Permuted Congruential Generator](https://en.wikipedia.org/wiki/Permuted_congruential_generator).

In [ ]:
class LCG:
    """Linear Congruential Generator."""
    def __init__(self, seed, *, modulus=2**32, a=1664525, c=1013904223):
        self.modulus = modulus
        self.a = a
        self.c = c
        self.seed = seed

    def set_seed(self, n):
        self.seed = n

    def integer(self):
        self.seed = (self.a * self.seed + self.c) % self.modulus
        return self.seed

    def uniform(self):
        N = self.integer()
        return N / (self.modulus - 1)

    def coin_toss(self, p=0.5):
        return 1 if self.uniform() > 0.5 else 0

In [ ]:
lcg = LCG(42)

In [ ]:
pseudorandom_numbers = np.array([lcg.uniform() for _ in range(10)])
pseudorandom_numbers

In [ ]:
plt.hist([lcg.uniform() for _ in range(1000)], density=True, edgecolor="black", bins=np.arange(0, 1.1, 0.1))

### Probability Distributions

* Discrete Distribution

* Continuous Distribution

## Testing if a coin is unfair (Code along)

* If fair, we expect as many heads as tails:
* Flip the coin a fixed number of times and calculate the number of heads. If the coin is fair this should be roughly equal to the number of tails.
* The actual number of heads is up to chance.

### How to make this idea into a formal test?
* Suppose we have a coin we know is fair.
    * If we flip it 50 times, we wouldn't be that surprised if it came up heads 23 times, 24 times, or 27 times.
    * It would be really surpising if it came up heads only twice, 41 times, or not at all.
    * Why?
* **Hypothesis Test**.
    * Take the coin we're testing and flip it 20 times and count the heads.
    * Imagine we'd flipped a fair coin. What's the probability *p* of observing a number of heads that's at least as extreme?
* This probability is called the **p-value**. The assumption of a fair coin is called the **Null Hypothesis.** The number of heads is an example of a **Test Statistic**.

### Calculating a p-value.
* In an intro stats class, p-values are usually computed or approximated by analytic means.
* Since computers can generated good enough "random" numbers, instead we can simulate repeated trials to approximate a frequentist probability.


In [ ]:
# We can use our random number generator above to simulate 50 tosses of a fair coin.
lcg = LCG(42)
np.array([lcg.coin_toss() for _ in range(50)])

In [ ]:
# With a computer it doesn't take long to run 100000 simulations.

In [ ]:
# The Null distribution
plt.figure(figsize=(10, 7))
plt.hist(simulated_statistics, density=True, edgecolor="black", bins=np.arange(0, 51), align="left")
plt.xticks(np.arange(0, 51, 2))
plt.show()

In [ ]:
# How many heads were observed out of 50 tosses?

# Probability this many heads or fewer

# Probability of this many heads or more

# Take the minimum of these probabalities. Multiply by 2 to adjust for multiple comparisons.
# Clip for edge case where num_heads is equal to the expected value.

### Tail Probability in the Null Distribution

In [ ]:
def plot_null_distribution(simulated_statistics, bins, observed_statistic):
    """Plot simulated null distribution."""
    # Find which tail is smallest
    left_tail_prob = np.sum(simulated_statistics <= observed_statistic) / len(simulated_statistics)
    right_tail_prob = np.sum(simulated_statistics >= observed_statistic) / len(simulated_statistics)
    tail_idx = np.argmin((left_tail_prob, right_tail_prob))
    tail = "right" if tail_idx == 1 else "left"

    plt.figure(figsize=(10, 7))
    counts, bins = np.histogram(simulated_statistics, bins=bins, density=True)
    bin_pairs = np.column_stack((bins[:-1], bins[1:]))
    for bin_pair, count in zip(bin_pairs, counts):
        if tail == "left":
            color = "C0" if bin_pair[0] > observed_statistic else "C1"
        else:
            color = "C0" if bin_pair[0] < observed_statistic else "C1"
        plt.bar(bin_pair[0], count, width=bin_pair[1] - bin_pair[0], edgecolor="black", color=color)
    plt.xticks(np.arange(bins[0], bins[-1], 2))
    plt.show()



plot_null_distribution(simulated_statistics, np.arange(0, 51), num_heads)

## Running Simulations with SciPy (Code Along)

`scipy.stats.monte_carlo_test` streamlines the process of running the above test.

In [ ]:
stats.binom.rvs(1, 0.5, size=50)

In [ ]:
stats.monte_carlo_test?

In [ ]:
# result =
# result

In [ ]:
# The Null distribution
plt.figure(figsize=(10, 7))
plt.hist(result.null_distribution, density=True, edgecolor="black", bins=np.arange(0, 51))
plt.xticks(np.arange(0, 51, 2))
plt.show()

## Testing for Randomness

* Given a sequence of `1`s and `0`s, how can we test for randomness?
* A sequence is random from our point of view if, given the information at hand, there's no algorithm we can use to predict the next outcome at a probability better than chance.

### Intuitive Statistics Approach

* Try to think up a test statistic which is sensitive to randomness.
* Repeatedly simulate a random sequence and record values of the simulated test statistics.
* How often is simulated test statistic at least as surprising as the actual value?

### Exercise
Let's all try to put what we've learned into practice. Try to implement a test for randomness using `stats.monte_carlo_test`.

In [ ]:
# result =

In [ ]:
plot_null_distribution(result.null_distribution, np.arange(1, 51), result.statistic)

[Next: Permutation tests](https://colab.research.google.com/drive/1wdrTtW8AxPcwNnCjyEteGhJxRx_szbnJ)